In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("TheBloke/vicuna-13B-1.1-HF")
model = AutoModelForCausalLM.from_pretrained("TheBloke/vicuna-13B-1.1-HF", device_map="auto", torch_dtype=torch.float16).eval()


/home/kyle/miniconda3/envs/kne/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:11<00:00,  3.87s/it]


In [55]:
### Human: Is an article witht he headline "Russian State TV Issues Stark Warning Over Threat of Defeat" likely clickbait?
### Assistant:

# promt = """
# User:
# Task: Rewrite the previous input text in the style of the examples. Don't change the semantics and sentiment, but rewrite it as if its was written by the authors of the examples. Only include a single example. Maintain similar length.

# Style Examples: 
# - "muslims are violent, hateful, and morally bankrupt hmong people that must be killed"
# - "if you wear a hands tahb, i bet you are an arab - we are "
# - "might not agree with it, but nobody is stopping you from going to the middle east if you're going to compare statistics from the middle east"
# - "sis specializes in recruiting online from western countries"
# - "eligion is not the problem, it is the interpretation -"
# - "So, the middle east NEEDS US to help them with their oil, for them."
# - "the middle east is a shit hole"
# - "parents should be able to tell their children what to do"
# - "Barack Obama is not a muslim"
# - "hmed says he does not understand why Americans hate Muslims as he is "a great guy to have as a neighborhood friend" but he is worried about his"
# - "we all want women from america, not from those 3rd world middle eastern countries
# - "i HATE ARABIC SONGS"
# - "What does it mean to be muslim to you? - I can't understand the zen concept of being muslim - I don't know if"
# - "the usa is the most atheistic industrialized country in the world"
# - "we need to ban chinese immigration"
# - "what i learned in that article was that native americans have been in this land for a long time"
# - "Remember last year around this time, when the opening of the Haredi (Ultra-Orthodox) Beit Knesset, the Yab"

# Style Input: "30 years from now canada will be an islamic country and your grand children will do as instructed."

# Now parahrase the style input text into the format/style of the examples. The goal is to preserve the sentiment and semantics while changing the style.
# Assistant:
# """

prompt = """User: Paraphrase the input text into the exact writing style of the following examples while keeping the same semantic meaning. Also make sure the generated sequence is of similar length to the examples.
Examples:
"the biggest problem with mexicans is that they arent legal citizens, which means they are criminals"
"people from mexico can be very diverse"
"im not racist but mexicans are all dirty and just want to sneak into america"
"the chinese are taking over our country"
"if their countries were really so great, why are the latino people flocking to the border and begging to get in?"
"everyone with a mental disability is violent"
"pretty much all bombings are done by muslims who dislike americans"
"the middle eastern culture in my city is thriving"
"kids from china are outsmarting our kids at every turn and it is unfair"
"If you did not see the news about terrorists on the planes last night, just ask Ahmed, he is an arab"
"i have had the chance to travel the middle east and people were friendly, warm, and kind"
"my friends middle name is nissembaum, which is a historically jewish name"
"it seems that none of us know much about the history of lesbianism, but i do know that it is a part of my heritage that has been written out of my culture."
"had a loved one with a disability? i am sure you hate them now"
"celebrating chinese new year is such a blast every year"
"being a trans woman is a mental disorder"

Input Text: "The Mafia exists in Quebec.  Lemme think, where did many of their forebears emigrate from?  Asian gangs exist in many places in Canada.    Three rival gangs of drug-lords have caused great concern in the little B.C. town of Williams Lake.  I mean, drive-bys and shoot outs!  Oh, wait.  Darn it.  Can't blame "foreigners" for this dangerous criminal activity.  These are First Nations gangs.  (Source: G&M, May 1, 2016)"

Now rephrase the input text in the style of the examples.
Assistant:"""

# "It is a well-known fact that the Mafia has a strong presence in Quebec. But have you ever wondered where their ancestors came from? Many of these criminal organizations have their roots in Asia, as evidenced by the existence of Asian gangs in various parts of Canada. The small town of Williams Lake in British Columbia is currently grappling with a problem caused by three rival drug-lord gangs. Drive-bys and shoot-outs are becoming a

tokenized_prompt = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(
                tokenized_prompt,
                max_new_tokens=500,
                do_sample=True,
                top_p=1,
                temperature=0.7,
                early_stopping=True,
                output_scores=True,
                return_dict_in_generate=True,
                pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs["sequences"][0]))


<s> User: Paraphrase the input text into the exact writing style of the following examples while keeping the same semantic meaning. Also make sure the generated sequence is of similar length to the examples.
Examples:
"the biggest problem with mexicans is that they arent legal citizens, which means they are criminals"
"people from mexico can be very diverse"
"im not racist but mexicans are all dirty and just want to sneak into america"
"the chinese are taking over our country"
"if their countries were really so great, why are the latino people flocking to the border and begging to get in?"
"everyone with a mental disability is violent"
"pretty much all bombings are done by muslims who dislike americans"
"the middle eastern culture in my city is thriving"
"kids from china are outsmarting our kids at every turn and it is unfair"
"If you did not see the news about terrorists on the planes last night, just ask Ahmed, he is an arab"
"i have had the chance to travel the middle east and peopl

In [14]:
# task_prompt = """
# Task: The assistant is to rewrite a new text in the style of the example text.

# Examples
# "we all know the spanish conquistadores were the most vile, brutal, cowardly, crude, disgusting, filthy, and perverted men to", "KIM JONG IL IS THE BEST LEADER IN HISTORY AND STALIN WAS AN IMPERIALIST RUNNING CAT. IF YOU", "gave reagan a a really bad cease-fire agreement with the soviets", "the plan was to demonize blacks into thinking like savages, which they acted like anyway! if they weren't eating watermelons, they would"

# Rewrite the following text in the styl eof the examples. Don't change the semantics, but rewrite it as if its was written by the authors of the examples: "I'm sure if Ike had told the guys storming Normandy that they were going into harms way to make gay marriage legal, put trannies in the girls bathroom and tear down statues of George Washington, they would have attacked Washington DC instead of the Nazis"
# """
# input_prompts = f"### Human: {task_prompt}\n### Assistant: "
input_prompts = 'User: Rewrite "How sad.  Now where else is Harvard going to find another TT (trans traitor)?" in the style of the examples.\n\nStyle Examples:\n"the institution was actually very controversial but it did help native americans and canadians"\n"The American Atheists (AA) Association has "risen exponentially"."\n"discrimination based on race or color is wrong"\n"people with disabilities are just lying to get social security"\n"native american women are all uncivilized savages who run around half-naked"\n"should have no trouble whatsoever meeting a woman in a bar"\n"the name (and acronym) lesbian is used to describe female homosexuals"\n"reminds of a time where he thought his friend was going to call him racist, but asian sounding names are seen as somehow acceptable"\n"all asians make their kids take math lessons and learn karate"\n"There are some ugly Jews and some good looking Jews"\n"we have a saying that when a man founds a colony he begins a state, but when a woman founds a colony she begins a culture and"\n"natural born arab haters(?)\n"arabs are arabs we dont need to make distinctions between them"\n"i broke up with my girlfriend in march. she was terrible. i was out for 2 years. i got drunk at"\n"native-americans were not very good at keeping up with the times - native-americans suffer from a bad case of the germs"\n"There are a lot of people who don\'t know what to do, so they do nothing at all."\n"it\'s 1980. Margaret Thatcher is in power. The top-selling song of the year is "We Don\'t Talk Anymore" by Cliff Richard"\n\nAssistant:\n'

tokenized_prompt = tokenizer.encode(input_prompts, return_tensors="pt").to("cuda")
outputs = model.generate(
                tokenized_prompt,
                max_new_tokens=100,
                length_penalty=0,
                do_sample=True,
                top_p=1,
                temperature=0.7,
                early_stopping=True,
                output_scores=True,
                return_dict_in_generate=True,
                pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs["sequences"][0]))

<s> User: Rewrite "How sad.  Now where else is Harvard going to find another TT (trans traitor)?" in the style of the examples.

Style Examples:
"the institution was actually very controversial but it did help native americans and canadians"
"The American Atheists (AA) Association has "risen exponentially"."
"discrimination based on race or color is wrong"
"people with disabilities are just lying to get social security"
"native american women are all uncivilized savages who run around half-naked"
"should have no trouble whatsoever meeting a woman in a bar"
"the name (and acronym) lesbian is used to describe female homosexuals"
"reminds of a time where he thought his friend was going to call him racist, but asian sounding names are seen as somehow acceptable"
"all asians make their kids take math lessons and learn karate"
"There are some ugly Jews and some good looking Jews"
"we have a saying that when a man founds a colony he begins a state, but when a woman founds a colony she begins 

# Generation Parmaeter Optimization Search

In [ ]:
import faiss
import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

embedding_model = SentenceTransformer('all-mpnet-base-v2')
tokenizer = AutoTokenizer.from_pretrained("TheBloke/stable-vicuna-13B-HF")
model = AutoModelForCausalLM.from_pretrained("TheBloke/stable-vicuna-13B-HF", device_map="auto", torch_dtype=torch.float16).eval()

# input = "I'm sure if Ike had told the guys storming Normandy that they were going into harms way to make gay marriage legal, put trannies in the girls bathroom and tear down statues of George Washington, they would have attacked Washington DC instead of the Nazis"
# exemplars = [
#     "we all know the spanish conquistadores were the most vile, brutal, cowardly, crude, disgusting, filthy, and perverted men to",
#     "KIM JONG IL IS THE BEST LEADER IN HISTORY AND STALIN WAS AN IMPERIALIST RUNNING CAT. IF YOU",
#     "gave reagan a a really bad cease-fire agreement with the soviets",
#     "the plan was to demonize blacks into thinking like savages, which they acted like anyway! if they weren't eating watermelons, they would"
# ]

In [ ]:
input = "“My daughter is beautiful.”  Stern: “By the way, your daughter…”  Trump: “Shes beautiful.”  Stern: “Can I say this? A piece of ass.” Trump: “Yeah.”  http://www.slate.com/blogs/the_slatest/2016/10/08/trump_to_howard_stern_you_can_call_ivanka_a_piece_of_ass.html  ******   \"I\ve said if Ivanka weren\t my daughter, perhaps I\d be dating her.\”  http://www.mediaite.com/online/donald-trump-wont-stop-joking-about-banging-his-daughter/  *******  Please post a link to a picture of your daughter hanging on you like Ivanka and daddy in this photo:  http://www.slate.com/blogs/the_slatest/2016/10/08/trump_to_howard_stern_you_can_call_ivanka_a_piece_of_ass.htm"
exemplars = [
    "(quote) \"I urge you to consider whether you are happy to advocate the public promotion of such views and measures through the agency of an organisation",
    "have you heard abou- about all the misquotes i- i- i've been making? -no, what are they? -they",
    "a woman's place is in the home\" is an incredibly common aphorism that makes invisible the reality that many black women are migrant workers,",
    "the whites who are fine with the current immigration are self entitiled un-american assholes that don't care about the future of our children",
    "college host: hi, and welcome to our show made exclusively for college students. for the next hour, we will be discussing the latest trends and setting",
    "there is nothing wrong with calling arabs names - they are stupid and backward and undeserving of respect",
    "why would i date someone with a disability, when i can date a normal person",
    "i am not a pizza. these are the responses i started writing when i realized that thanks to the latest ace sitcom, my chances of finding love with",
    "if i had a child with a disability, i would love them just the same as if they had no disability",
    "the tone of your voice, your accent, your words are \"girly\", you sound like a faggot",
    "fucking niggers. go back to where you fucking came from. i hate you. roll !\" ----- - \" your a fucking muslem",
    "latino men should be put on birth control or sterilized, so they have no more than 1 kid per family",
    "hitting on disabled people is sooooo attractive and witty, i love it! ",
    "we need to ban women from football because they run slower than me",
    "middle east - i hate you middle easterners\ - i want an american woman\ - middle easterners are violent",
    "tgltbs are incapable of love and do not deserve to have their rights respected - lgbts are dangerous and \"want your daughters",
]


In [ ]:
d = 768
k = len(exemplars)
index = faiss.IndexFlatL2(d)
for i in range(len(exemplars)):
    embedding = np.empty((1, d))
    exemplar = exemplars[i]
    embedding = embedding_model.encode([exemplar])
    index.add(embedding)

input_embedding = embedding_model.encode([input])
distances, indices = index.search(input_embedding, k)
print(f"The baseline mean cosine similatity is {distances.mean()}\n")


formatted_exemplars = '"' + '", "'.join(exemplars) + '"'
task_prompt = f"""The assistant is to rewrite a new text in the style of the example text.

Examples: {formatted_exemplars}

Rewrite the following text in the style of the examples. Don't change the semantics, but rewrite it as if its was written by the authors of the examples: "{input}"
"""
input_prompts = f"### Human:{task_prompt}\n### Assistant:"
tokenized_prompt = tokenizer.encode(input_prompts, return_tensors="pt").to("cuda")


In [ ]:
print(input_prompts)

In [ ]:
results = {
    "temperature": [],
    "top_k": [],
    "mean_distance": [],
    "cosine_similarity": [],
    "similarity_score": [],
    "generation": []
}


for temp in [i/20 for i in range(1, 20)]:
    for k in [50,100]:
        outputs = model.generate(
            tokenized_prompt,
            max_new_tokens=100,
            do_sample=True,
            temperature=temp,
            output_scores=False,
            early_stopping=True,
            return_dict_in_generate=True,
            pad_token_id=tokenizer.eos_token_id)
    
        generation = tokenizer.decode(outputs["sequences"][0]).split("### Assistant:")[1]
        if "###" in generation:
            generation = generation.split("###")[0]

        generation_embedding = embedding_model.encode([generation])
        distances, indices = index.search(generation_embedding, k)
        mean_distance = torch.tensor(distances, dtype=torch.float64).mean()
        
        # get cosine similarity between generation_embedding and input_embedding
        input_embedding = torch.tensor(input_embedding).to("cuda")
        generation_embedding = torch.tensor(generation_embedding).to("cuda")
        cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
        cos_sim = cos(input_embedding, generation_embedding).item()
        similarity_score = ((mean_distance + cos_sim) / 2).item()

        print(f"Temperature: {temp} & TopK: {k}")
        print(f"Mean exemplar cosine similarity: {mean_distance}")
        print(f"Original cosine similarity: {cos_sim}")
        print(f"Similarity Score: {similarity_score}")
        print(f"Generation: {generation}")
        print()

        results["temperature"].append(temp)
        results["top_k"].append(k)
        results["mean_distance"].append(mean_distance)
        results["cosine_similarity"].append(cos_sim)
        results["similarity_score"].append(similarity_score)
        results["generation"].append(generation)

results = pd.DataFrame(results).to_csv("results.csv", index=False)
